In [1]:
import pickle

In [2]:
import os
from tqdm import tqdm
from typing import Dict
from collections import defaultdict
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def numLines(fname):
    with open(fname) as f:
        for i, l in enumerate(f):
            pass
    return i + 1
def loadData(filename, max_points):
    file_len = numLines(filename)
    f = open(filename, 'r')
    inputs = []
    outputs = []
    for i in tqdm(range(file_len)):
        if i == max_points:
            break
        line = f.readline()
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        inputs.append(line[0])
        outputs.append(line[1])
    data = {'inputs': inputs, 'outputs': outputs}
    return data
        
def load_entity_strings(filename):
    with open(filename) as f:
        lines = f.read().splitlines()
    return lines

def get_entity_wd_id_dict(filename):
    out = {}
    f = open(filename, 'r')
    for line in f:
        if line[-1] == '\n':
            line = line[:-1]
        line = line.split('\t')
        out[line[1]] = line[0]
    return out
    

def create_filter_dict(data) -> Dict[str, int]:
    filter_dict = defaultdict(list)
    for input, output in zip(data["inputs"], data["outputs"]):
        filter_dict[input].append(output)
    return filter_dict

def getAllFilteringEntities(input, filter_dicts):
    entities = []
    splits = ['train', 'test', 'valid']
    for s in splits:
        entities.extend(filter_dicts[s][input])
    return list(set(entities))

def wikidata_link_from_id(id):
    uri = 'https://www.wikidata.org/wiki/' + id
    return uri

In [3]:
dataset_name = 'codex-m'
entity_strings = load_entity_strings(os.path.join("data", dataset_name, "entity_strings.txt"))

In [4]:
entity_strings_set = set(entity_strings)
len(entity_strings_set)

17050

In [5]:
data = {}
splits = ['train', 'valid', 'test']
for split in splits:
    data[split] = loadData(os.path.join('data', dataset_name, split + '.txt'), -1)

100%|███████████████████████████████████████████████████████| 20622/20622 [00:00<00:00, 673547.40it/s]


In [6]:
filter_dicts = {}
splits = ['train', 'valid', 'test']
for split in splits:
    filter_dicts[split] = create_filter_dict(data[split])

In [7]:
# fname = 'scores.pickle'
fname = 'scores/scores_codexm3.pickle'
# fname = 'scores/scores_temp.pickle'
# fname = 'scores_500_base_trie.pickle'
scores_data = pickle.load(open(fname, 'rb'))

In [8]:
predictions_scores_dicts = []
for string_arr, score_arr in tqdm(zip(scores_data['prediction_strings'], scores_data['scores'])):
    ps_pairs = [(p,s) for p,s in zip(string_arr, score_arr)]
    ps_pairs = list(set(ps_pairs)) # while sampling, duplicates are created
    # remove predictions that are not entities
    ps_dict_only_entities = defaultdict(list)
    for ps in ps_pairs:
        if ps[0] in entity_strings_set:
            ps_dict_only_entities[ps[0]] = ps[1]
    predictions_scores_dicts.append(ps_dict_only_entities)

20622it [00:02, 7342.98it/s]


In [9]:
max([len(predictions_scores_dicts[x]) for x in range(500)])

348

In [10]:
id = 2
print(scores_data['input_strings'][id], scores_data['target_strings'][id])
# print(scores_data['scores'][2])
predictions_scores_dicts[id]

predict tail: fukuzawa yukichi | occupation | critic


defaultdict(list,
            {'businessperson': -3.2318945,
             'human rights activist': -6.3340797,
             'translator': -3.6245623,
             'prosaist': -7.266861,
             'economist': -4.110162,
             'teacher': -2.6061964,
             'political scientist': -2.6477232,
             'scientist': -6.5660505,
             'politician': -1.5181198,
             'professor': -2.036954,
             'educator': -8.009469,
             'linguist': -5.683258,
             'opinion journalist': -5.255703,
             'model': -8.190271,
             'lawyer': -6.8618584,
             'literary critic': -5.509801,
             'geopolitician': -6.9307632,
             'diplomat': -4.9691353,
             'writer': -1.95469,
             'university teacher': -5.495364,
             'philosopher': -2.254507,
             'journalist': -2.8599358,
             'poet': -6.220251,
             'pedagogue': -2.836358,
             'tarento': -6.8111076,
         

In [11]:
import numpy as np
predictions_filtered = []
head_num_filter = 0
tail_num_filter = 0
hits_at_all = 0
count = 0
for i in tqdm(range(len(predictions_scores_dicts))):
    ps_dict = predictions_scores_dicts[i].copy()
    target = scores_data['target_strings'][i]
    inputs = scores_data['input_strings'][i]
    prediction_strings = ps_dict.keys()
    if target in prediction_strings:
        original_score = ps_dict[target]
    # get filtering entities
    filtering_entities = getAllFilteringEntities(inputs, filter_dicts)
    if len(filtering_entities) <= 1:
        count += 1
    if 'head' in inputs:
        head_num_filter += len(filtering_entities)
    else:
        tail_num_filter += len(filtering_entities)
    for ent in filtering_entities:
        if ent in ps_dict:
            ps_dict[ent] = -float("inf")
    if target in prediction_strings:
        ps_dict[target] = original_score
        hits_at_all += 1
    # softmax for scores
    names_arr = []
    scores_arr = []
    for k, v in ps_dict.items():
        names_arr.append(k)
        scores_arr.append(v)
    scores_arr = np.array(scores_arr)
#     scores_arr = softmax(scores_arr)
    for name, score in zip(names_arr, scores_arr):
        ps_dict[name] = score
    predictions_filtered.append(ps_dict)
print(head_num_filter/len(predictions_filtered), tail_num_filter/len(predictions_filtered))
print(hits_at_all/len(predictions_filtered))

100%|█████████████████████████████████████████████████████████| 20622/20622 [00:03<00:00, 5373.74it/s]

491.37426049849677 3.0085345747260206
0.38235864610610026


In [13]:
count, len(predictions_filtered)

(3471, 20622)

In [25]:
count = {}
reciprocal_ranks = 0.0
k_list = [1,3,10]
for k in k_list:
    count[k] = 0
num_small_arrs = 0
count2 = 0
for i in range(len(predictions_filtered)):
    target = scores_data['target_strings'][i]
    ps_dict = predictions_filtered[i]
    ps_sorted = sorted(ps_dict.items(), key=lambda item: -item[1])
    if len(ps_dict) == 0:
        preds = []
    else:
        preds = [x[0] for x in ps_sorted]
    if target in preds:
        rank = preds.index(target) + 1
        reciprocal_ranks += 1./rank
    for k in k_list:
        if target in preds[:k]:
            count[k] += 1
    if len(preds) < 10 and target not in preds:
        num_small_arrs += 1
    if target in preds:
        count2 += 1
for k in k_list:
    hits_at_k = count[k]/len(predictions_filtered)
    print('hits@{}'.format(k), hits_at_k)
print('mrr', reciprocal_ranks/len(predictions_filtered))
print(num_small_arrs/len(predictions_filtered), 'were <10 length preds array without answer')

hits@1 0.20924255649306567
hits@3 0.29483076326253516
hits@10 0.361846571622539
mrr 0.26066608206281355
0.07491998836194355 were <10 length preds array without answer


In [13]:
count2/len(predictions_filtered)

0.34099505382601103

In [126]:
id = 4
inputs = scores_data['input_strings'][id]
preds = predictions_filtered[id]
preds = sorted(preds.items(), key=lambda item: -item[1])
target = scores_data['target_strings'][id]

In [76]:
print(inputs, 'Target:', target)
preds[:10], target

predict tail: Valenciennes | isLocatedIn | Target: Nord-Pas-de-Calais


([('Île-de-France', 0.3138781066564107),
  ('Lower_Silesian_Voivodeship', 0.13227073610334242),
  ('Nord-Pas-de-Calais', 0.09675851055651631),
  ('Valenciennes', 0.07590361486184467),
  ('Lists_of_World_Heritage_Sites_in_Europe', 0.061758891746029325),
  ('Centre_region,_France', 0.052148043026076424),
  ('Lesser_Poland_Voivodeship', 0.044005159013402746),
  ('Pays_de_la_Loire', 0.04173769409652669),
  ('Paris', 0.03586650379301719),
  ('Quebec', 0.030035223333408627)],
 'Nord-Pas-de-Calais')

In [482]:
# only head/tails
count = 0
for id in range(60,120, 2):
    inputs = scores_data['input_strings'][id]
    preds = predictions_filtered[id]
    preds = sorted(preds.items(), key=lambda item: -item[1])
    target = scores_data['target_strings'][id]
    pred1 = preds[0][0]
    if pred1 == target:
        print(int(id/2), inputs, pred1)
        count += 1
'count', count

32 predict tail: ali kazemaini | birthplace | tehran
34 predict tail: ashta, maharashtra | instance of | human settlement
37 predict tail: roy shaw 0 | instance of | human being
40 predict tail: t. canby jones | has surname | jones (family name)
45 predict tail: naveen kumar | instance of | human being
46 predict tail: barlow respiratory hospital | host country | united states of america
48 predict tail: camiling | office held by head of government | mayor
51 predict tail: hazel soan | instance of | human being
52 predict tail: efrain herrera | sport played | association football
53 predict tail: oluf munck | instance of | human being
54 predict tail: thomas gilchrist | instance of | human being
58 predict tail: desmoplastic fibroma | subclass of | fibroma


('count', 12)

In [419]:
%%html
print("<a href='your_url_here'>Showing Text</a>")


In [364]:
e2wdid['pakistan']

'Q4121082'

In [174]:
sequences = ['english', 'english language', 'french']
t = Trie(sequences)

In [178]:
t.get('x')

[]